In [1]:
import json
import copy

In [2]:
# pretty prints
from pprint import pprint

In [3]:
# creating directory for json configs
import os

if not os.path.isdir("gobot"):
    os.mkdir("gobot")

<img src="https://static.tildacdn.com/tild6461-6138-4365-a139-383131346165/ipavlov_logo__.png" alt="teacher_forcing" width=50%/>

In [4]:
import deeppavlov

/Users/yaroina-kente/deeppavlov-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:841)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:841)>
[nltk_data] Error loading perluniprops: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:841)>
[nltk_data] Error loading nonbreaking_prefixes: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:841)>
2018-07

<style type="text/css">
#notebook-container{width: 210mm;!important;
 font-family: inherit;
 font-size:120%;
} 

**NOTE**: "go_bot" model trains faster on a CPU, so let's ignore existing GPUs:

In [5]:
!export CUDA_VISIBLE_DEVICES=""
!echo "cuda visible devices = '"$CUDA_VISIBLE_DEVICES"'"

cuda visible devices = ''


# Hybrid goal-oriented bot

Dialog bots are categorized into two types:

1. **goal-oriented models **

    (those who have to achieve some kind of a goal in the end of conversation: 
     - restaurant and flight booking,
     - customer support service,
     - etc.);
 
2. **chit-chat models **

   (those who chat just for fun, the longer bot speaks with you the better, example:
    - "replica" mobile application).

We will only dive into goal-oriented task specification.

![go bot architecture 0](scheme000.png)

A classical dialog system consists of:

1. **Natural Language Understanding component (NLU)**

 that is intended to "understand" human and represent it's "understanding" in a machine readable format. 

 It takes an utterance text as input and converts to a dialog "frame".

 "Frame" may consist of:
   - domain value (domain is some kind of "a type of dialogs");
   - intent value (intent of current human utterance: "welcome_message", "asking_weather", etc.);
   - entity slots (entities are mentioned by human "location", "time", etc.).

2. **Dialogue Manager component (DM)**

 that is intended to decide what to respond. 
    
 It takes a filled by NLU frame and outputs action (it isn't a final text, it is a label). 
    
 For example, there may be actions: "say_welcome", "say_goodbye", "ask_location", "give_weather", etc.

3. **Natural Language Generation component(NLG)**

 that is intended to convert action to an actual text response representation.
 
 For example, "say_goodbye" -> "You are welcome!".

## NLU 

![go bot architecture 0](scheme001.png)

Let's consider a dialog system with NLU component that consists of a single Named Entity Recognition 
    component (or NER).

One of the previous tutorials introduced deeppavlov NER model and showed how to use it.

## DM & NLG

The tutorial is focused on how to implement
 - Dialogue Manager and
 - Natural Language Generator.

### Dataset

We will train the chatbot on a [Dialog State Tracking Chellenge 2](http://camdial.org/~mh521/dstc/) data.

Let's download it first.

In [6]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2Version2DatasetReader

data = DSTC2Version2DatasetReader().read(data_path="tmp/my_download_of_dstc2")

2018-07-05 19:22:04.152 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-trn.jsonlist]
2018-07-05 19:22:05.322 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-val.jsonlist]
2018-07-05 19:22:05.876 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-tst.jsonlist]


`DSTC2Version2DatasetReader` downloaded the needed data and saved to disk.

`DialogDatasetIterator` took the data as input and transformed it to batches.

In [7]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

batches_generator = DialogDatasetIterator(data, seed=1443, shuffle=True)\
                                         .gen_batches(batch_size=4, data_type='train')

-------------
-------------
   Let's take a closer look at a batch content:

In [8]:
batch = batches_generator.__next__()

Each batch is a tuple of two elements:
  - list of x's and
  - list of y's

In [9]:
x_batch, y_batch = batch

`x_batch` (and `y_batch`) consists of 4 samples. This is because `batch_size` was 4.

In [10]:
len(x_batch)

4

One sample is a dialog. This is how one turn from a dialog looks like:

In [11]:
dialog_id = 0
dialog = [(x, y) for x, y in zip(x_batch[dialog_id], y_batch[dialog_id])]

turn_id = 0
print("----{}th turn----".format(turn_id)) 
pprint(dialog[turn_id], indent=8)

----0th turn----
(       {'intents': [], 'prev_resp_act': None, 'text': ''},
        {       'act': 'welcomemsg',
                'text': 'Hello, welcome to the Cambridge restaurant system. '
                        'You can ask for restaurants by area, price range or '
                        'food type. How may I help you?'})


This is how the whole dialog looks (printing `'text'` parts of `x_batch` and `y_batch`):

In [12]:
for turn in dialog:
    x, y = turn
    print('::', x['text']) 
    print('>>', y['text'], '\n')

:: 
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you? 

:: traditional
>> api_call area="dontcare" food="traditional" pricerange="dontcare" 

:: traditional
>> I am sorry but there is no traditional restaurant that matches your request. 

:: italian food
>> What part of town do you have in mind? 

:: south
>> api_call area="south" food="italian" pricerange="dontcare" 

:: south
>> Pizza hut cherry hinton is a nice place in the south of town serving tasty italian food. 

:: phone number
>> The phone number of pizza hut cherry hinton is 01223 323737. 

:: post code
>> The post code of pizza hut cherry hinton is C.B 1, 7 D.Y. 

:: thank you good bye
>> You are welcome! 



### HowTo: DeepPavlov configs

DeepPavlov uses json configs as a tool to configure data pipelines.

Config has the following sections:
     
 - **dataset_reader**
   - configuration of dataset reader component (is responsible for data download and saving to disk);
  
 - **dataset_iterator**
   - configuration of dataset iterator component (is responsible for making batches (sequences) of data that will be further fed to pipe components);
  
 - **metadata**
   - extra info (urls for data download and telegram configuration);

 - **train**
   - training process configuration (size of batches, number of training epochs, etc.);
   
 - **chainer**
   - specifies data flow (which components are run and in what order);

Let's construct a simple config that builds a dictionary of input sample tokens.

In [13]:
vocab_config = {}

- **dataset_reader** -- configuration of dataset reader component (that is responsible for data download and saving to disk)

In [14]:
dstc2_reader_comp_config = {
    'name': 'dstc2_v2_reader',
    'data_path': 'dstc2_v2'
}

In [15]:
vocab_config['dataset_reader'] = dstc2_reader_comp_config

- **dataset_iterator** -- configuration of dataset iterator component (that is responsible for making batches (sequences) of data that will be further fed to pipe components)

In [16]:
dialog_iterator_comp_config = {
    'name': 'dialog_iterator'
}

In [17]:
vocab_config['dataset_iterator'] = dialog_iterator_comp_config

- **metadata** -- some extra info
     - **metadata.download** -- a list of data which should be downloaded in order for config to work

In [18]:
dstc2_download_config = {
    'url': 'http://lnsigo.mipt.ru/export/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2_v2'
}

In [19]:
vocab_config['metadata'] = {}
vocab_config['metadata']['download'] = [
    dstc2_download_config
]

- **train** -- training process configuration
     
We don't need to train anything now, just build (fit on whole dataset once) a dictionary, so "train" section is empty.

In [20]:
vocab_config['train'] = {}

 - **chainer** specifies data flow:
 
     - **chainer.in** -- is a list of input sample names (one data sample might consist of several variables);
     - **chainer.in_y** -- is a list of input label names (each sample might have labels of different kind);
     - **chainer.out** -- is a list of output prediction names (usually has the same length as "chainer.in_y");
     
X is only an utterance here.

Y is empty (we don't need to train the dictionary like neural networks)

There is no prediction for the config, nothing to predict.

In [21]:
vocab_config['chainer'] = {}
vocab_config['chainer']['in'] = ['utterance']
vocab_config['chainer']['in_y'] = []
vocab_config['chainer']['out'] = []

- **chainer**
     - **chainer.pipe** -- is a list of consequently run components. This is the place where you specify in which order and what kind of data will be fed to components. 
     
Our pipe consists of one component -- "default_vocab".

##### HowTo: Component config

Component configs are always just a part of a global model config (described above).

Config for any component contains the following **_required_** parameters:
 - **name** -- registered name of a component (it is a link to python component implementation)
 - **save_path** -- path to save the component (sometimes is not needed, for example, for tokenizers)
 - **load_path** -- path to load the component (sometimes is not needed, for examples, for tokenizers)

and the following **_optional_** parameters:
 - **id** -- reference name for the component
 - **ref** -- "id" of a component that was previously initialized. It can be used instead of "name".
 - **fit\_on** -- a list of data fields to fit on (it calls \_\_fit\_\_ method of the component)
 - **in** -- a list of data fields that are inputs during inference (prediction)
 - **out** -- a list of data fields that are outputs during inference (prediction)
 
 
"default_vocab" component also has it's on unique parameters:
 - level -- on which level to operate ('token' level and 'char' (character) level are available)
 - tokenizer -- if input is a string, then it will be tokenized by the tokenizer, _optional parameter_

In [22]:
vocab_comp_config = {
    'name': 'default_vocab',
    'save_path': 'vocabs/token.dict',
    'load_path': 'vocabs/token.dict',
    'fit_on': ['utterance'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'},
    'main': True
}

In [23]:
vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]

In [24]:
json.dump(vocab_config, open("gobot/vocab_config.json", 'wt'))

To download "dstc2_v2" dataset use `deeppavlov.deep_download` script (you have to do it only once):

In [25]:
from deeppavlov.download import deep_download # it is called "deep" in honor of "Deep Pavlov"

deep_download(['--config', 'gobot/vocab_config.json'])

2018-07-05 19:22:49.620 INFO in 'deeppavlov.download'['download'] at line 142: Downloading...
2018-07-05 19:22:49.635 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): lnsigo.mipt.ru
2018-07-05 19:22:49.646 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://lnsigo.mipt.ru:80 "GET /export/datasets/dstc2_v2.tar.gz HTTP/1.1" 200 506300
2018-07-05 19:22:49.649 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://lnsigo.mipt.ru/export/datasets/dstc2_v2.tar.gz to /Users/yaroina-kente/DeepPavlov/download/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 7.80MB/s]
2018-07-05 19:22:49.722 INFO in 'deeppavlov.core.data.utils'['utils'] at line 149: Extracting /Users/yaroina-kente/DeepPavlov/download/dstc2_v2.tar.gz archive into /Users/yaroina-kente/DeepPavlov/download/dstc2_v2
2018-07-05 19:22:49.800 INFO in 'deeppavlov.download'['download'] at line 144: 
Download successful!


All data and models are saved to root of deeppavlov module + `../download` (`DEEPPAVLOV_ROOT/../download/`).

In [26]:
dstc2_v2_path = deeppavlov.__path__[0] + '/../download/dstc2_v2'

Data was downloaded to `dstc2_v2_path`:

In [27]:
# The command will only work for linux, do not panic otherwise -- it isn't something crucially important.
# You can further just comment bash commands.
!echo "> ls $dstc2_v2_path"
!ls $dstc2_v2_path

> ls /Users/yaroina-kente/DeepPavlov/deeppavlov/../download/dstc2_v2
dstc2-templates.txt dstc2-tst.jsonlist  resto.sqlite
dstc2-trn.jsonlist  dstc2-val.jsonlist


Let's build our vocabulary.

In [28]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config

train_evaluate_model_from_config("gobot/vocab_config.json")

2018-07-05 19:22:55.422 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-07-05 19:22:56.339 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-07-05 19:22:57.214 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-07-05 19:22:58.236 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 19:22:58.319 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 19:22:58.328 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162:

{"valid": {"eval_examples_count": 575, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 576, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01"}}


Vocabulary was built on data and saved to disk.

`save_path = 'vocabs/token.dict'` and component files are saved to `DEEPPAVLOV_ROOT/../download/vocabs/token.dict`.

In [29]:
vocabs_path = deeppavlov.__path__[0] + '/../download/vocabs'

In [30]:
!echo "> ls $vocabs_path"
!ls $vocabs_path

> ls /Users/yaroina-kente/DeepPavlov/deeppavlov/../download/vocabs
token.dict


This is the content of the saved "token.dict":

In [31]:
!echo "> head $vocabs_path/token.dict"
!head $vocabs_path/token.dict

> head /Users/yaroina-kente/DeepPavlov/deeppavlov/../download/vocabs/token.dict
cheap	391
restaurant	1179
any	259
south	306
address	781
phone	819
number	825
thank	998
you	1055
good	891


##### Using trained component

We can use built vocabulary by initializing it with `build_model_from_config`.

We need to add `in` and `out` to component configuration ( to know what are inputs and outputs during prediction ) :
 - **in** -- a list of data fields that are inputs during inference (prediction)
 - **out** -- a list of data fields that are outputs during inference (prediction)

In [32]:
vocab_comp_config['in'] = ['utterance']
vocab_comp_config['out'] = ['utterance_token_indices']

vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]
vocab_config['chainer']['out'] = ['utterance_token_indices']

In [33]:
from deeppavlov.core.commands.infer import build_model_from_config

model = build_model_from_config(vocab_config)

2018-07-05 19:23:10.119 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]


Model expects a list of samples (batch) as input.

In [34]:
model(['hi'])

[141]

### Model `gobot_dstc2_simple`

Now let's train a simple goal-oriented bot:

In [35]:
from deeppavlov.download import deep_download
from deeppavlov.core.commands.train import train_evaluate_model_from_config
from deeppavlov.core.commands.infer import build_model_from_config

"dataset_reader", "dataset_iterator" and "metadata" will be the same as for vocabulary only.

In [36]:
simple_config = {}

simple_config['dataset_reader'] = dstc2_reader_comp_config
simple_config['dataset_iterator'] = dialog_iterator_comp_config
simple_config['metadata'] = {}
simple_config['metadata']['download'] = [
    dstc2_download_config
]

X here is a dict `'x'` containing context 'text', 'intents', db_result', 'prev_resp_act'

Y here is a dict `'y'` containing response 'act' and 'text'

Prediction `'y_predicted'` here will be only 'text'

In [37]:
simple_config['chainer'] = {}
simple_config['chainer']['in'] = ['x']
simple_config['chainer']['in_y'] =['y']
simple_config['chainer']['out'] = ['y_predicted']

The bot consists (`pipe` section) of two components:
- **`default_vocab`** (or DefaultVocabulary) component that 

    - remembers all tokens from user utterances. 
    - `DefaultVocabulary.__call__` method inputs batch of tokens and outputs their indeces.

Vocabulary component will be the same as before, but let's add reference to component using `id` 
- **id** -- reference name for the component

In [38]:
vocab_comp_config = {
    'name': 'default_vocab',
    'id': 'token_vocab',
    'load_path': 'vocabs/token.dict',
    'save_path': 'vocabs/token.dict',
    'fit_on': ['x'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'}
}

Adding vocabulary to chainer:

In [39]:
simple_config['chainer']['pipe'] = []
simple_config['chainer']['pipe'].append(vocab_comp_config)

- **`go_bot`** (or GoalOrientedBot) component that
    - calls `slot_filler` that for user utterance outputs mentioned slots 
        (for example, "i want cheap food" -> {'pricerange': 'cheap'})
    - updates dialog state with `tracker` (DialogStateTracker)
        
          (for example, if old state was {'location': 'north'}, 
          and current slots are {'pricerange': 'cheap'}, 
          then new dialog state will be {'location': 'north', 'pricerange': 'cheap'})
    - converts user utterance in string format (`x`) to tokens with `tokenizer`

          (for example, "hi, i want some cheap food" -> ['hi', ',', 'i', 'want', 'some', 'cheap', 'food'])
    - then embeds the tokens with bag-of-words using `bow_embedder`(if not None) and `word_vocab`

          (for example, "cheap" -> [1, 0, 0, 0, .., 0])
    - embeds the utterance with continuous `embedder` (if not None) as a mean of embeddings of utterance tokens
        
          (for example, "i" -> [0.1231, 0.23423, .., 0.03489])
    - concatenates embeddings and passes it as an input to a recurrent neural network (RNN)
    - trains RNN (with LongShortTermMemory (LSTM) as a core graph) that outputs an action label
    - loads templates (mapping from labels to string) using `template_path` and `template_type` and converts action label to string
        
          (for example, "bye_msg" -> "You are welcome!")
    - fills result string with slot values from dialog state
        
          (for example, if
           dialog state is equal to {'pricerange': 'cheap'}
           and output string is "There are no restaurants in a #pricerange pricerange"
           then the result response will be "There are no restaurants in a cheap pricerange")

In [40]:
bot_comp_config = {
    'name': 'go_bot',
    'in': ['x'],
    'in_y': ['y'],
    'out': ['y_predicted'],
    'word_vocab': None,
    'bow_embedder': {"name": "bow"},
    'embedder': None,
    'slot_filler': None,
    'template_path': 'dstc2_v2/dstc2-templates.txt',
    'template_type': 'DualTemplate',
    'database': None,
    'api_call_action': 'api_call',
    'network_parameters': {
      'load_path': 'gobot_dstc2_simple/model',
      'save_path': 'gobot_dstc2_simple/model',
      'dense_size': 64,
      'hidden_size': 128,
      'learning_rate': 0.002,
      'attention_mechanism': None
    },
    'tokenizer': {'name': 'stream_spacy_tokenizer',
                  'lowercase': False},
    'tracker': {'name': 'featurized_tracker',
                'slot_names': ['pricerange', 'this', 'area', 'food', 'name']},
    'main': True,
    'debug': False
}

This is how we use vocabulary by reference:

In [41]:
bot_comp_config['word_vocab'] = '#token_vocab'

Announcing slot filler component.
We assume that slot filler is already trained, and use it by referencing it's config.

In [42]:
slot_filler_comp_config = {
    'config_path': deeppavlov.__path__[0] + '/../deeppavlov/configs/ner/slotfill_dstc2.json'
}

Adding slot filler to bot component:

In [43]:
bot_comp_config['slot_filler'] = slot_filler_comp_config

Adding `bot_comp_config` to `pipe`:

In [44]:
simple_config['chainer']['pipe'].append(bot_comp_config)

Neural network (in the bot) is trained in epochs, and needs data in the form of batches.

That is why we are now filling "train" section with training parameters.

- **train** -- training process configuration
     - **train.batch_size** is a number of samples in a batch (feeded to the network during one training step)
     - **train.epochs** is a number of iterations over dataset during training
     - **train.log_every_n_batches** and **train.log_every_n_epochs** control frequency of logging messages
     - **train.metrics** is a list of metrics used to validate our performance
     - **train.val_every_n_batches** and **train.val_every_n_epochs** describes how often we calculate metrics on `valid` data split
     - **train.validation_patience** is a number of epochs without metric improvement on `valid` data that we are able to endure =)

In [45]:
simple_bot_train_config = {
    'batch_size': 4,
    'epochs': 2,
    'log_every_n_batches': -1,
    'log_every_n_epochs': 1,
    'metrics': ['per_item_dialog_accuracy'],
    'val_every_n_epochs': 1,
    'validation_patience': 20
}

In [46]:
simple_config['train'] = simple_bot_train_config

In [47]:
json.dump(simple_config, open("gobot/simple_config.json", 'wt'))

`train.epochs` is set to '2' for now, if you intend to train a smarter model, you should increase it (a range from 10 to 200 epochs is recommended).

In [48]:
deep_download(['--config', slot_filler_comp_config['config_path']])

2018-07-05 19:23:54.428 INFO in 'deeppavlov.download'['download'] at line 142: Downloading...
2018-07-05 19:23:54.442 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): lnsigo.mipt.ru
2018-07-05 19:23:54.451 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://lnsigo.mipt.ru:80 "GET /export/deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-07-05 19:23:54.454 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://lnsigo.mipt.ru/export/deeppavlov_data/slotfill_dstc2.tar.gz to /Users/yaroina-kente/DeepPavlov/slotfill_dstc2.tar.gz
100%|██████████| 641k/641k [00:00<00:00, 6.72MB/s]
2018-07-05 19:23:54.557 INFO in 'deeppavlov.core.data.utils'['utils'] at line 149: Extracting /Users/yaroina-kente/DeepPavlov/slotfill_dstc2.tar.gz archive into /Users/yaroina-kente/DeepPavlov/download
2018-07-05 19:23:54.586 INFO in 'deeppavlov.download'['download'] at line 144: 
Download successful!


In [ ]:
train_evaluate_model_from_config("gobot/simple_config.json")

2018-07-05 19:23:56.87 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-07-05 19:23:57.53 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-07-05 19:23:58.123 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-07-05 19:23:58.683 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 19:23:58.776 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 19:23:58.797 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



2018-07-05 19:24:00.57 WARNING in 'tensorflow'['tf_logging'] at line 126: From /Users/yaroina-kente/DeepPavlov/deeppavlov/models/ner/network.py:215: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

2018-07-05 19:24:02.690 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model]


INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 19:24:02.747 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 19:24:04.253 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 19:24:04.257 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 19:24:04.262 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 19:24:04.265 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 19:24:06.875 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork` from saved]
2018-07-05 19:24:06.884 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 303: [loading parameters from /Users/yaroina-kente/D

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_simple/model


2018-07-05 19:24:06.936 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_simple/model


Let's comminicate with the resulting bot. "exit" message initiates end of dialogue.

In [50]:
model = build_model_from_config(simple_config)

2018-07-05 16:16:50.482 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 16:16:50.493 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-07-05 16:16:50.505 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-07-05 16:16:52.593 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model]


INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 16:16:52.639 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 16:16:53.455 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 16:16:53.456 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 16:16:53.458 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 16:16:53.459 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 16:16:54.840 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork` from saved]
2018-07-05 16:16:54.841 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 303: [loading parameters from /Users/yaroina-kente/D

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_simple/model


2018-07-05 16:16:54.876 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_simple/model


In [52]:
model(['hi, i want some cheap italian food in the north of town'])

2018-07-05 16:17:09.89 WARNING in 'deeppavlov.skills.go_bot.bot'['bot'] at line 345: No database specified.
2018-07-05 16:17:09.90 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 346: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [54]:
model(['thanks, bye'])

2018-07-05 16:17:21.564 WARNING in 'deeppavlov.skills.go_bot.bot'['bot'] at line 345: No database specified.
2018-07-05 16:17:21.565 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 346: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [55]:
model.reset() # resetting dialog context to start a new one

In [56]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

2018-07-05 16:17:42.854 WARNING in 'deeppavlov.skills.go_bot.bot'['bot'] at line 345: No database specified.
2018-07-05 16:17:42.855 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 346: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


>> Sorry there is no italian restaurant in the north of town.
:: I want cheap food


2018-07-05 16:17:55.22 WARNING in 'deeppavlov.skills.go_bot.bot'['bot'] at line 345: No database specified.
2018-07-05 16:17:55.24 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 346: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


>> Sorry there is no italian restaurant in the north of town.
:: food


2018-07-05 16:18:13.79 WARNING in 'deeppavlov.skills.go_bot.bot'['bot'] at line 345: No database specified.
2018-07-05 16:18:13.80 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 346: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


>> Sorry there is no italian restaurant in the north of town.
:: exit


The model couldn't fill some slots. For example, #address, #phone, #postcode of a restaurant couldn't be inferred from user utterance. 

A list of available restaurants is required.

### Model `gobot_dstc2_db`

Now let's now add a database with restaurants and train a new model:

Initializing new config:

In [57]:
db_config = copy.deepcopy(simple_config)

db_config['chainer']['pipe'] = []

Creating database component config:

In [58]:
db_comp_config = {
    'name': 'sqlite_database',
    'id': 'restaurant_database', 
    'save_path': 'dstc2_v2/resto.sqlite',
    'primary_keys': ['name'],
    'table_name': 'mytable'
}

Adding vocab and database components to pipe:

In [59]:
db_config['chainer']['pipe'].append(vocab_comp_config)
db_config['chainer']['pipe'].append(db_comp_config)

Initializing bot component config:

In [60]:
bot_with_db_comp_config = copy.deepcopy(bot_comp_config)

**WARNING:** Do no forget to change `load_path` and `save_path` in neural network configuration when 
             training a new modification. Otherwise previous model's files will be overwritten.

In [61]:
bot_with_db_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_db/model'
bot_with_db_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_db/model'

Adding database to bot component config:

In [62]:
bot_with_db_comp_config['database'] = '#restaurant_database'

Addind bot component to pipe:

In [63]:
db_config['chainer']['pipe'].append(bot_with_db_comp_config)

In [64]:
json.dump(db_config, open("gobot/db_config.json", 'wt'))

The new model now updates dialog state not only with entity values mentioned by user ("i want cheap food" -> {'pricerange': 'cheap'}), but also with restaurant info taken from sql database of restaurants.

Model has a special action `api_call_action`, which initiates a request to sql database with current dialog state and thus receives info of a single matching restaurant.

So now such slots as #address, #phone and #postcode can be filled in bot responses.

In [65]:
train_evaluate_model_from_config("gobot/db_config.json")

2018-07-05 16:19:02.591 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-07-05 16:19:03.67 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-07-05 16:19:03.263 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-07-05 16:19:03.712 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 16:19:03.756 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 16:19:03.759 WARNING in 'deeppavlov.core.models.serializable'['serializa

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 16:19:05.286 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 16:19:06.17 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 16:19:06.18 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 16:19:06.21 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 16:19:06.24 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 16:19:07.436 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 55: [initializing `GoalOrientedBotNetwork` from scratch]
2018-07-05 16:22:16.311 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 315: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.3578}, "time_spent": "0:03:09"}}


2018-07-05 16:22:53.599 INFO in 'deeppavlov.core.commands.train'['train'] at line 343: New best per_item_dialog_accuracy of 0.3688
2018-07-05 16:22:53.600 INFO in 'deeppavlov.core.commands.train'['train'] at line 345: Saving model
2018-07-05 16:22:53.603 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_db/model]
2018-07-05 16:22:53.787 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 297: [saving parameters to /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_db/model.json]


...

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 16:25:27.416 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 16:25:28.182 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 16:25:28.184 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 16:25:28.186 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 16:25:28.187 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 16:25:29.282 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork` from saved]
2018-07-05 16:25:29.283 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 303: [loading parameters from /Users/yaroina-kente/D

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_db/model


2018-07-05 16:25:29.302 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_db/model
2018-07-05 16:25:29.369 INFO in 'deeppavlov.core.commands.train'['train'] at line 174: Testing the best saved model


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4325}, "time_spent": "0:00:38"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4247}, "time_spent": "0:00:40"}}


In [66]:
model = build_model_from_config(db_config)

2018-07-05 16:29:27.275 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 16:29:27.280 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-07-05 16:29:27.282 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 57: Loading database from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/resto.sqlite.
2018-07-05 16:29:27.290 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-07-05 16:29:27.299 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-07-05 16:29:30.728 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 4

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 16:29:30.803 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 16:29:32.492 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 16:29:32.495 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 16:29:32.503 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 16:29:32.517 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 16:29:35.432 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork` from saved]
2018-07-05 16:29:35.434 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 303: [loading parameters from /Users/yaroina-kente/D

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_db/model


2018-07-05 16:29:35.483 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_db/model


In [67]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: something Italian
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: cheap and Italian, where to go
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: hi
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: exit


### Model `gobot_dstc2_emb`

Now let's train a goal-oriented bot with fasttext embeddings:

**NOTICE:** YOU NEED TO CONSTRUCT A NEW CONFIG YOURSELF

Initalizing new config:

In [68]:
emb_config = copy.deepcopy(db_config)

emb_config['chainer']['pipe'] = []
emb_config

Adding vocab component to chainer pipe:

In [69]:
emb_config['chainer']['pipe'].append(vocab_comp_config)

Initalizing embedder component:

In [70]:
embedder_comp_config = {
    'id': 'my_embedder',
    'name': 'fasttext',
    'load_path': 'embeddings/dstc2_fastText_model.bin',
    'save_path': 'embeddings/dstc2_fastText_model.bin',
    'dim': 100
}

In [71]:
# TODO: add embedder component to chainer pipe
emb_config['chainer']['pipe'].append(embedder_comp_config)

Initializing bot component config:

In [72]:
bot_with_embedder_comp_config = copy.deepcopy(bot_with_db_comp_config)

bot_with_embedder_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_emb/model'
bot_with_embedder_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_emb/model'

In [73]:
# TODO: add #my_embedder to bot_with_embedder_comp_config
bot_with_embedder_comp_config['embedder']= '#' + embedder_comp_config['id']

In [74]:
# TODO: add bot_with_embedder_comp_config to chainer pipe
emb_config['chainer']['pipe'].append(bot_with_embedder_comp_config)

These are download urls for new required data:

In [75]:
embedder_required_data = {
    'url': 'http://lnsigo.mipt.ru/export/deeppavlov_data/embeddings/dstc2_fastText_model.bin',
    'subdir': 'embeddings'
}

In [80]:
# TODO: add embedder download info to emb_config['metadata']
#emb_config['metadata'] = {}
emb_config['metadata']['download'] = [
    embedder_required_data
]

In [81]:
json.dump(emb_config, open("gobot/emb_config.json", 'wt'))

As far as we are now using embeddings, we added a file named `dstc2_fastText_model.bin` to `metadata.download` section. 

Let's run data loading again.

In [82]:
deep_download(['--config', 'gobot/emb_config.json'])

2018-07-05 17:12:24.702 INFO in 'deeppavlov.download'['download'] at line 142: Downloading...
2018-07-05 17:12:24.712 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): lnsigo.mipt.ru
2018-07-05 17:12:24.739 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://lnsigo.mipt.ru:80 "GET /export/deeppavlov_data/embeddings/dstc2_fastText_model.bin HTTP/1.1" 200 800203996
2018-07-05 17:12:24.742 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://lnsigo.mipt.ru/export/deeppavlov_data/embeddings/dstc2_fastText_model.bin to /Users/yaroina-kente/DeepPavlov/download/embeddings/dstc2_fastText_model.bin
100%|██████████| 800M/800M [07:19<00:00, 1.82MB/s] 
2018-07-05 17:19:44.662 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): lnsigo.mipt.ru
2018-07-05 17:19:44.684 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://lnsigo.mipt.ru:80 "

In [83]:
train_evaluate_model_from_config("gobot/emb_config.json")

2018-07-05 17:19:48.370 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-07-05 17:19:49.585 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-07-05 17:19:50.802 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-07-05 17:19:51.520 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 17:19:51.724 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 17:19:51.729 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['f

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 17:20:03.498 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 17:20:05.175 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 17:20:05.182 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 17:20:05.187 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 17:20:05.199 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 17:20:08.386 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 55: [initializing `GoalOrientedBotNetwork` from scratch]
2018-07-05 17:23:04.136 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 315: Updating global step, learning rate = 0.00200

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.3934}, "time_spent": "0:02:56"}}


2018-07-05 17:23:40.897 INFO in 'deeppavlov.core.commands.train'['train'] at line 343: New best per_item_dialog_accuracy of 0.3946
2018-07-05 17:23:40.897 INFO in 'deeppavlov.core.commands.train'['train'] at line 345: Saving model
2018-07-05 17:23:40.899 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 49: [saving model to /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_emb/model]
2018-07-05 17:23:41.85 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 297: [saving parameters to /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_emb/model.json]


...

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 17:26:02.805 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 17:26:03.605 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 17:26:03.606 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 17:26:03.609 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 17:26:03.610 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 17:26:04.609 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork` from saved]
2018-07-05 17:26:04.610 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 303: [loading parameters from /Users/yaroina-kente/D

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_emb/model


2018-07-05 17:26:04.631 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_emb/model
2018-07-05 17:26:04.863 INFO in 'deeppavlov.core.commands.train'['train'] at line 174: Testing the best saved model


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4511}, "time_spent": "0:00:36"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4416}, "time_spent": "0:00:35"}}


In [84]:
model = build_model_from_config(emb_config)

2018-07-05 17:28:52.77 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 17:28:52.82 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 69: [loading embeddings from `/Users/yaroina-kente/DeepPavlov/download/embeddings/dstc2_fastText_model.bin`]
2018-07-05 17:28:58.229 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-07-05 17:28:58.245 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/tag.dict]
2018-07-05 17:29:04.97 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 40: [loading model from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model]


INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 17:29:04.145 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 17:29:05.892 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 17:29:05.903 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 17:29:05.931 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 17:29:05.934 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 17:29:09.433 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 52: [initializing `GoalOrientedBotNetwork` from saved]
2018-07-05 17:29:09.438 INFO in 'deeppavlov.skills.go_bot.network'['network'] at line 303: [loading parameters from /Users/yaroina-kente/D

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_emb/model


2018-07-05 17:29:09.474 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/gobot_dstc2_emb/model


In [85]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: food
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: please
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: restaurant
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: bye
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: exit


## Appendix _(optional)_

### Additional materials

- [DataFest Presentation (RU)](https://docs.google.com/presentation/d/1PBPQp-wgQ6aRbm3MsuyGYB_TVg2c7Bf89lhdhbMtC2k)
- [Video Lecture "Hybrid dialog bot" (RU)](http://www.youtube.com/watch?v=JJCO7eWCy-M&t=331m19s)
- [Video Lecture "What's inside a dialog system?" (RU)](http://www.youtube.com/watch?v=JJCO7eWCy-M&t=259m55s)

### Model `gobot_dstc2_full`

Now let's train a very smart goal-oriented bot that uses an attention mechanism over input embeddings 

(see https://medium.com/syncedreview/a-brief-overview-of-attention-mechanism-13c578ba9129 for more details):

Initializing new config:

In [94]:
emb_config = json.load(open("gobot/emb_config.json", 'rt'))

full_config = copy.deepcopy(emb_config)
full_config['chainer']['pipe'] = [
    vocab_comp_config,
    db_comp_config,
    embedder_comp_config
]

Initializing bot component config:

In [95]:
bot_with_emb_comp_config = emb_config['chainer']['pipe'][-1]
bot_with_attn_comp_config = copy.deepcopy(bot_with_emb_comp_config)

bot_with_attn_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_full/model'
bot_with_attn_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_full/model'

Adding attention mechanism to bot:

In [96]:
attention_mechanism_config = {
    'action_as_key': True,
    'depth': 3,
    'hidden_size': 32,
    'max_num_tokens': 100,
    'projected_align': False,
    'type': 'cs_bahdanau'
}

In [97]:
bot_with_attn_comp_config['network_parameters']['attention_mechanism'] = attention_mechanism_config

Adding bot component to pipe:

In [98]:
full_config['chainer']['pipe'].append(bot_with_attn_comp_config)

In [99]:
json.dump(full_config, open("gobot/full_config.json", 'wt'))

In [100]:
train_evaluate_model_from_config("gobot/full_config.json")

2018-07-05 18:51:40.279 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-trn.jsonlist]
2018-07-05 18:51:41.291 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-val.jsonlist]
2018-07-05 18:51:44.447 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-tst.jsonlist]
2018-07-05 18:51:44.911 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 18:51:45.65 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 150: [saving vocabulary to /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 18:51:45.92 WARNING in 'deeppavlov.core.models.serializable'['serializab

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 18:51:57.478 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 18:51:58.529 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 18:51:58.530 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 18:51:58.534 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 18:51:58.536 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 18:51:58.539 ERROR in 'deeppavlov.core.common.params'['params'] at line 123: Exception in <class 'deeppavlov.skills.go_bot.bot.GoalOrientedBot'>
Traceback (most recent call last):
  File "/Users/yaroina-kente/DeepPavlov/deeppavlov/core/common/params.py", line 117, in 

AttributeError: 'NoneType' object has no attribute 'dim'

In [101]:
model = build_model_from_config(full_config)

2018-07-05 18:52:05.363 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 162: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/vocabs/token.dict]
2018-07-05 18:52:05.370 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-07-05 18:52:05.372 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 57: Loading database from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/resto.sqlite.
2018-07-05 18:52:05.375 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 69: [loading embeddings from `/Users/yaroina-kente/DeepPavlov/download/embeddings/dstc2_fastText_model.bin`]
2018-07-05 18:52:12.609 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 94: [loading vocabulary from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/word.dict]
2018-07-05 18:52:12.663 INFO in 'deeppavlov.core.data.si

INFO:tensorflow:Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model


2018-07-05 18:52:15.457 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from /Users/yaroina-kente/DeepPavlov/download/slotfill_dstc2/model
2018-07-05 18:52:17.162 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-07-05 18:52:17.167 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 69: [loading templates from /Users/yaroina-kente/DeepPavlov/download/dstc2_v2/dstc2-templates.txt]
2018-07-05 18:52:17.173 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 72: 46 templates loaded
2018-07-05 18:52:17.175 INFO in 'deeppavlov.skills.go_bot.bot'['bot'] at line 96: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-07-05 18:52:17.177 ERROR in 'deeppavlov.core.common.params'['params'] at line 123: Exception in <class 'deeppavlov.skills.go_bot.bot.GoalOrientedBot'>
Traceback (most recent call last):
  File "/Users/yaroina-kente/DeepPavlov/deeppavlov/core/common/params.py", line 117, in 

AttributeError: 'NoneType' object has no attribute 'dim'

In [102]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: hi
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: cambridge
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: exit


### Another way of training and infering a component

Let's build response token vocabulary, but do it without using deeppavlov scripts (without `train_evaluate_model_from_config` and `build_model_from_config`).

In [103]:
from deeppavlov.core.data.vocab import DefaultVocabulary
from deeppavlov.dataset_readers.dstc2_reader import DSTC2Version2DatasetReader
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

Initializing a `DefaultVocabulary` class:

In [104]:
y_vocab = DefaultVocabulary(level='token', 
                            load_path='vocabs/y_token.dict', # path is relative to DEEPPAVLOV_ROOT/../download/ 
                            save_path='vocabs/y_token.dict',
                            tokenizer=lambda s_batch: [s.split() for s in s_batch])

Important methods of any trained component are:

   - **\_\_init\_\_(self, *args, *\*kwargs)**
     - intializes a class instance

   - **fit(self, data, *args)** or **train_on_batch(self, batch, *args)**
     - fits on full data or makes one training step on a batch of data

   - **\_\_call\_\_(self, batch, \*\*kwargs)**
     - makes prediction (or infers) for each sample in a batch

Getting batches of data:

In [105]:
data = DSTC2Version2DatasetReader().read(data_path="tmp/my_download_of_dstc2")
data_samples = DialogDatasetIterator(data, seed=1443, shuffle=True).get_instances(data_type='all')
x_list, y_list = data_samples

2018-07-05 18:53:00.786 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-trn.jsonlist]
2018-07-05 18:53:04.724 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-val.jsonlist]
2018-07-05 18:53:05.214 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 214: [loading dialogs from tmp/my_download_of_dstc2/dstc2-tst.jsonlist]


Building vocabulary using y batches:

In [106]:
y_vocab.fit(y_list)

Infering from (using) built vocabulary:

In [107]:
y_vocab(['is', 'the', 'of', 'restaurant', 'hi'])

[43, 3, 26, 5, 0]

To call a model `x_vocab(batch)` is the same as to call a \_\_call\_\_ method `x_vocab.__call__(batch)`!

In [108]:
y_vocab(['hi']) == y_vocab.__call__(['hi']) == [141]

False